In [1]:
import re
#REGEX VARIABLES:
VAR_PREFIX = r'\$|_|[a-zA-Z]'
VAR_SUFFIX = fr'{VAR_PREFIX}|\d'
VAR = fr'(?:{VAR_PREFIX})(?:{VAR_SUFFIX})*'
CHR_EL = r"""(?:[^'"\\]|(?:\\n)|(?:\\\\)|(?:\\\')|(?:\\\")|(?:\\r)|(?:\\t))"""
CHR = fr"'(?:{CHR_EL})'"
STR = fr'"(?:{CHR_EL})*"'
NUM = fr"(?:-?\d+)|(?:{CHR})"
VAR_SINGLE = fr'(?P<list>(?P<l_var>{VAR})\s*\[\s*(?P<index>{NUM})\s*\])|(?P<var>{VAR})'
PARAM = f"(?P<v>{VAR})|(?P<n>{NUM})|(?P<s>{STR})"

In [2]:
import operator
from functools import partial
#MEMORY VARS:
R0 = 10
OPS = {'add':operator.add,'sub':operator.sub,'div':operator.floordiv,'mul':operator.mul,'divmod':divmod,'mod':operator.mod}

class BFCompiler(object):
    def __init__(self, inp=None):
        self.code, self.p, self.inp = '', 0, inp
        self.vars, self.lists, self.next_mem = {},{},R0+1
        self.on_end = []
        
    def run(self, code):
        for line in code:
            #var
            varm = re.match(f'var(?P<vars> (?:{VAR_SINGLE}))+',line)
            if varm:
                variables = re.finditer(VAR_SINGLE, line[4:])
                for v in variables:
                    self.declare(v.group('var') or v.group('l_var'), v.group('index'))
                continue
            #SET, INC, DEC
            setm = re.match(f'set (?P<var_name>{VAR}) (?:(?P<v>{VAR})|(?P<n>{NUM}))', line) 
            if setm:
                x = setm.group('var_name')
                y = setm.group('v') or int(setm.group('n'))
                #if vbl not in self.vars: self.declare(vbl) ?
                self.copy(x, y)
                continue
            inc_dec = re.match(f'(?P<op>inc|dec) (?P<var_name>{VAR}) (?:(?P<v>{VAR})|(?P<n>{NUM}))', line)
            if inc_dec:
                x = inc_dec.group('var_name')
                y = inc_dec.group('v') or int(inc_dec.group('n'))
                if inc_dec.group('op') == 'dec': self.dec_by(x,y) 
                else: self.inc_by(x,y)
                continue
            #math
            reg_ops = '|'.join(OPS.keys())
            math_r = fr'(?P<op>{reg_ops}) (?:(?P<av>{VAR})|(?P<an>{NUM})) (?:(?P<bv>{VAR})|(?P<bn>{NUM})) (?P<c>{VAR})( (?P<d>{VAR}))?'
            math = re.match(math_r,line)
            if math:
                op = math.group('op')
                a = math.group('av') or int(math.group('an'))
                b = math.group('bv') or int(math.group('bn'))
                c,d = math.group('c'), math.group('d')
                #for v in [c,d]:
                    #if v: self.declare(v) 
                self.math(op, a,b,c,d)
                continue
            #lset, lget
            lset = re.match(f'lset (?P<lst>{VAR}) (?:(?P<iv>{VAR})|(?P<in>{NUM})) (?:(?P<vv>{VAR})|(?P<vn>{NUM}))',line)
            if lset:
                lst = lset.group('lst')
                i = lset.group('iv') or int(lset.group('in'))
                v = lset.group('vv') or int(lset.group('vn'))
                self.lset(lst,i,v)
                continue
            lget = re.match(f'lget (?P<lst>{VAR}) (?:(?P<iv>{VAR})|(?P<in>{NUM})) (?P<v>{VAR})',line)
            if lget:
                lst = lget.group('lst')
                i = lget.group('iv') or int(lget.group('in'))
                self.lget(lst,i,lget.group('v'))
                continue
            #a2b b2a
            a2b = re.match(f'a2b (?:(?P<av>{VAR})|(?P<an>{NUM})) (?:(?P<bv>{VAR})|(?P<bn>{NUM})) (?:(?P<cv>{VAR})|(?P<cn>{NUM})) (?P<d>{VAR})',line)
            if a2b:
                a = a2b.group('av') or int(a2b.group('an'))
                b = a2b.group('bv') or int(a2b.group('bn'))
                c = a2b.group('cv') or int(a2b.group('cn'))
                d = a2b.group('d')
                self.a2b(a,b,c,d)
                continue
            b2a = re.match(f'b2a (?:(?P<av>{VAR})|(?P<an>{NUM})) (?P<b>{VAR}) (?P<c>{VAR}) (?P<d>{VAR})',line)
            if b2a:
                a = b2a.group('av') or int(b2a.group('an'))
                b,c,d = b2a.group('b'),b2a.group('c'),b2a.group('d')
                self.b2a(a,b,c,d)
            #IO
            read = re.match(f'read (?P<v>{VAR})',line)
            if read:
                self.mv(read.group('v')).cmd(',')
                continue
            write = re.match(f'msg (?P<args>((\s)*((?:{STR})|(?:{VAR}))(\s)*)+)',line)
            if write:
                tokens = [i.group(0) for i in re.finditer(f'({STR})|({VAR})',write.group('args'))]
                self.msg(tokens)
                continue
            #cmp
            cmp = re.match(f'cmp (?:(?P<av>{VAR})|(?P<an>{NUM})) (?:(?P<bv>{VAR})|(?P<bn>{NUM})) (?P<c>{VAR})',line)
            if cmp:
                a = cmp.group('av') or int(cmp.group('an'))
                b = cmp.group('bv') or int(cmp.group('bn'))
                c = cmp.group('c')
                self.cmp(a,b,c)
                continue
            #flow
            def end_if(self, pos):
                    self.mv(pos)
                    self.cmd(']')
            ifeq = re.match(f'ifeq (?P<a>{VAR}) (?:(?P<bv>{VAR})|(?P<bn>{NUM}))',line)
            if ifeq:
                for i in range(2): self.clear(self.next_mem + i)
                b = ifeq.group('bv') or int(ifeq.group('bn'))
                self.math('sub',ifeq.group('a'),b,self.next_mem)
                self.mv(self.next_mem)
                self.cmd('>+<[>-<[-]]') #next_mem + 1 holds 1 if a-b 0, else 0. ____ 0 >1 _ or ___ 0 >0 _
                self.mv(self.next_mem + 1)
                #pointing to next_mem + 1, but thinks at next_mem
                self.cmd('[')
                self.clear(self.next_mem + 1) #no need to clear in alt case b/c looks like 00
                self.on_end.append(partial(end_if, self, self.next_mem + 1))
                continue
            ifneq = re.match(f'ifneq (?P<a>{VAR}) (?:(?P<bv>{VAR})|(?P<bn>{NUM}))', line)
            if ifneq:
                b = ifneq.group('bv') or int(ifneq.group('bn'))
                self.clear(self.next_mem)
                self.math('sub',ifneq.group('a'),b,self.next_mem)
                self.mv(self.next_mem)
                self.cmd('[')
                self.clear(self.next_mem)
                self.on_end.append(partial(end_if,self,self.next_mem))
                continue
            wneq = re.match(f'wneq (?P<a>{VAR}) (?:(?P<bv>{VAR})|(?P<bn>{NUM}))', line)
            def end_while(self,a,b,x):
                self.clear(x)
                self.math('sub',a,b,x)
                self.mv(x)
                self.cmd(']')
            if wneq:
                x,self.next_mem = self.next_mem, self.next_mem + 1
                a,b = wneq.group('a'), wneq.group('bv') or int(wneq.group('bn'))
                self.clear(x)
                self.math('sub',a,b,x)
                self.mv(x)
                self.cmd('[')
                self.on_end.append(partial(end_while,self,a,b,x))
            if re.match(f'end', line): self.on_end.pop()() #ends a loop or if statement <3
        return self.code
        
    def cmp(self, a, b, c):
        #0 1 0 a b 0         0
        for i,v in enumerate([0,1,0,a,b,0,0]): self.copy(self.next_mem + i, v)
        self.math("sub",a,b,self.next_mem+6)
        self.mv(self.next_mem + 6) #point to a-b
        self.cmd("[") #runs if a != b
        self.mv(self.next_mem+3).cmd('+>+<') #increment a and b
        self.cmd('#[') #runs if a is not 255
                
        self.cmd(">>>[-]<<<") #Clear a-b to show execution
        self.cmd("[->-[>]<<]") #point to next_mem + 2 if a > b, else next_mem + 3
        self.cmd("<[->>>>+<<<<]") #a > b: write 1 to n5
        self.cmd("<[-<>>>>>-<<<<<]") #a < b: write -1 to n5
        self.cmd(">>>[-]") #return to a #CHANGED FROM 6 AND ADDED CLEAR
            
        self.cmd(']')
        self.mv(self.next_mem+6) #move to nm6
        self.cmd('[[-]<+>]') #writes 1 to nm+5, clears nm+6 if a = 255
        self.cmd("]")
        self.copy(c, self.next_mem + 5,pos=True)
        for i in range(7): self.clear(self.next_mem + i) #clear work
        
            
                
    def msg(self, tokens):
        x = self.next_mem
        for t in tokens:
            if re.match(STR, t):
                subs = {'\\n':'\n','\\t':'\t','\\r':'\r'}
                for c in re.findall(CHR_EL, t[1:-1]): 
                    if c in subs: c = subs[c]
                    self.copy(x,ord(c)).mv(x).cmd('.')
            else:
                self.copy(x, t).mv(x).cmd('.')
        return self.clear(x)
                
    def a2b(self,a,b,c,d):
        #'a' 'b' 'c' -> d. Note: 0 = 48 ... 9 = 57
        _a,_b,_c = R0-8,R0-9,R0-10
        self.vars['~a'] = _a
        self.vars['~b'] = _b
        self.vars['~c'] = _c
        for i in [_a,_b,_c]: self.clear(i)
        self.copy(_a,a).copy(_b,b).copy(_c,c)
        for i in [_a,_b,_c]: self.dec_by(i, 48)
        self.math('mul',100,'~a','~a')
        self.math('mul',10,'~b','~b')
        self.inc_by(_c,'~a')
        self.inc_by(_c,'~b')
        self.copy(d,'~c')
        for i in [_a,_b,_c]: self.clear(i)
        return self
    
    def b2a(self,a,b,c,d):
        self.math('divmod',a,100,b,c)
        self.math('divmod',c,10,c,d)
        for i in [b,c,d]: self.inc_by(i,48)
        return self
    
    def lset(self, lst, i, v):
        '''x[y] = z, or lst[i] = v'''
        l = self.lists[lst]
        x,t0,t1 = l['x'],l['t0'],l['t1']
        for var in [x,t0,t1]: self.clear(var)
        self.copy(t1,i).copy(t0,v)
        self.mv(x)
        return self.cmd('>>[[>>]+[<<]>>-]+[>>]<[-]<[<<]>[>[>>]<+<[<<]>-]>[>>]<<[-<<]')
    
    def lget(self, lst, i, x):
        '''x = lst[i]'''
        l = self.lists[lst]
        y,t0,t1 = l['x'],l['t0'],l['t1']
        for var in [x,y,t0,t1]: self.clear(var)
        self.copy(t1,i)
        self.mv(y).cmd('>>[[>>]+[<<]>>-]+[>>]<[<[<<]>+<')
        return self.mv(x).inc().mv(y).cmd('>>[>>]<-]<[<<]>[>[>>]<+<[<<]>-]>[>>]<<[-<<]')
        
    def declare(self, var, length=None):
        if length is None: 
            self.vars[var] = self.next_mem
            self.next_mem += 1
        else:
            self.lists[var] = {
                'x':self.next_mem,
                't0':self.next_mem + 1,
                't1':self.next_mem + 2
            }
            self.next_mem += 3 + 2*int(length)
        return self
            
    def math(self, op, a, b, c, d=None):
        if type(a) is int and type(b) is int:
            if op == 'divmod':
                _c,_d = OPS[op](a,b)
                return self.copy(c,_c).copy(d,_d)
            return self.copy(c, OPS[op](a,b))
        #now to do the math!
        if op == 'add': 
            self.clear(R0-1).inc_by(R0-1,a).inc_by(R0-1,b)
            return self.clear(c).copy(c,R0-1,pos=True).clear(R0-1)
        elif op == 'sub': 
            self.clear(R0-1).copy(R0-1,a).dec_by(R0-1,b)
            return self.clear(c).copy(c,R0-1,pos=True).clear(R0-1)
        if op == 'mul':
            _c = R0-3
            self.mv(_c).cmd("[-]+") #set R3 to 1
            for y in [a,b]: #Multiply R3 by a and b, rep'd as y
                if type(y) is int:
                    self.copy(R0-4, y)
                    y = R0-4
                self.clear(R0-1).clear(R0-2)
                self.mv(_c).code += '['
                self.mv(R0-2).inc().mv(_c).dec().code += ']'
                self.mv(R0-2).code += '['
                self.mv(y).code+='['
                #check if c should be x
                self.mv(_c).inc().mv(R0-1).inc().mv(y).dec().code += ']'
                self.mv(R0-1).code+='['
                self.mv(y).inc().mv(R0-1).dec().code += ']'
                self.mv(R0-2).dec().code += ']'
                if type(y) is int: self.clear(y)
            return self.clear(c).copy(c,_c,pos=True).clear(_c)
        #divmod
        for i in range(1,10): self.clear(R0-i) #clear R1-R9
        self.copy(R0-8,b)
        self.copy(R0-5,b)
        self.copy(R0-7,a)
        #must check for denominator of 1
        self.mv(R0-8).cmd('-[[-]') #code runs only if b != 1
        self.mv(R0-9).cmd('+')
        self.mv(R0-7)
        self.cmd('[->+>-[>+>>]>[+[-<+>]>+>>]<<<<<<]') #>0 n d-n%d n%d n/d
        self.mv(R0-8).cmd(']')
        self.mv(R0-9).cmd('-[+')
        self.clear(R0-4)
        self.copy(R0-3,a)
        self.mv(R0-9).cmd(']')
        if op == 'div':
            #implement later
            return self.copy(c,R0-3,pos=True)
        elif op == 'mod':
            return self.copy(c,R0-4,pos=True)
        elif op == 'divmod':
            return self.copy(c,R0-3,pos=True).copy(d,R0-4,pos=True)
        
    def cmd(self,s):
        self.code+=s
        return self
    
    def inc_by(self, x, y, pos=False):
        if type(x) is str: x = self.vars[x]
        if type(y) is int and not pos:
            self.mv(x).code += "+"*y
            return self
        elif not pos: y = self.vars[y]
        #x = x + y
        #temp0[-]
        #y[x+temp0+y-]
        #temp0[y+temp0-]
        self.clear(R0)
        self.mv(y).code += '['
        self.mv(x).inc().mv(R0).inc().mv(y).dec()
        self.code+= ']'
        self.mv(R0).code += '['
        self.mv(y).inc().mv(R0).dec().code+=']'
        return self
    
    def dec_by(self, x, y):
        if type(x) is str: x = self.vars[x]
        if type(y) is int:
            self.mv(x).code += "-"*y
            return self
        else: y = self.vars[y]
        #temp0[-]
        #y[x-temp0+y-]
        #temp0[y+temp0-]
        self.clear(R0)
        self.mv(y).code += '['
        self.mv(x).dec().mv(R0).inc().mv(y).dec()
        self.code+= ']'
        self.mv(R0).code += '['
        self.mv(y).inc().mv(R0).dec().code+=']'
        return self
        
    def copy(self, x, y, pos=False):
        """Returns code needed to copy a value from cell y to cell x, starting at a given cell"""
        """x = y"""
        if type(x) is str: x = self.vars[x]
        if type(y) is int and not pos:
            self.clear(cell=x).code += "+"*int(y)
            return self
        elif not pos: y = self.vars[y]
        t = R0
        self.clear(t).clear(x).mv(y)
        self.code += '['
        self.mv(x).inc()
        self.mv(t).inc()
        self.mv(y).dec()
        self.code += ']'
        self.mv(t)
        self.code += '['
        self.mv(y).inc()
        self.mv(t).dec()
        self.code += ']'
        return self
    def dec(self):
        self.code += '-'
        return self
    def inc(self):
        self.code += '+'
        return self
    def mv(self, dest):
        if type(dest) is str: dest = self.vars[dest]
        self.code += (('<' if self.p > dest else '>')*abs(dest-self.p))
        self.p = dest
        return self
    def clear(self, cell=None):
        if cell is None: cell = self.p
        self.mv(cell)
        self.code += '[-]'
        return self

In [3]:
import re

def kcuf(code):
    code = remove_comments(code).split('\n')
    code = [re.sub(fr'(?P<str>{STR})',' \g<str> ',i.strip()) for i in code if i]
    def decode(m):
        char = m.group(0)[1:-1]
        subs = {'\\n':'\n','\\t':'\t','\\r':'\r'}
        if char in subs: char = subs[char]
        return str(ord(char))
    code = [re.sub(CHR,decode,c) for c in code]
    code = [clean(c) for c in code]
    code = validate(code)

    b = BFCompiler()
    res = b.run(code)
    print(res)
    
def remove_comments(code):
    comments_gone = ""
    def hide_quotes(cd):
        res,count = "",0
        for c in cd:
            if c == '"':
                count += 1
                res += '"'
            else: res += c if count%2==0 else 's'
        return res
    comments = re.finditer(r'(\/\/|((^|\n)rem )|#|--)[^\n]*(?=\n|$)',hide_quotes(code))
    spans = [i.span() for i in comments]
    first = 0
    for span in spans:
        comments_gone += code[first:span[0]]
        code += '\n'
        first = span[1]
    comments_gone += code[first:]
    return comments_gone
def clean(s):
    return remove_whitespace(fix_nums(lower(s)))
    
def remove_whitespace(s):
    return re.sub(r' +',' ',s)
    
def fix_nums(s):
    if s[:3] != "msg":
        return re.sub(r'(?: -?\d+)', lambda m: " " + str(int(m.group(0)) % 256), s)
    return s

def lower(s):
    res = ""
    for i,c in enumerate(s):
        if len(re.findall("(?<!\\\\)(\"|')",s[:i])) % 2 is 0:
            res += c.lower()
        else: res+=c
    return res

def validate(code):
    #Check ends match and illegal proc stuff, as well as properly closed '',[],""
    count, in_proc = 0, False
    def match_parens(l):
        count = 0
        for c in l:
            if c == '[': count+=1
            elif c==']': count-=1
            if count < 0: return False
        return count == 0
    for line in code:
        if len(re.findall(r"(?<![^\\]\\)'",line)) % 2 != 0:
            raise Exception("Unclosed '' pair.")
        if len(re.findall(r'(?<![^\\]\\)"',line)) % 2 != 0:
            raise Exception('Unclosed "" pair.' + " LINE: " + line)
        if not match_parens(line):
            raise Exception('Unclosed [] pair.')
        if re.match('(?:ifeq)|(?:ifneq)|(?:wneq)|(?:proc)',line): count += 1
        elif re.match('end',line): 
            count -= 1
            if count is 0: in_proc = False
        if re.match('(?:proc)',line):
            if in_proc: raise Exception('Nested procedures.')
            in_proc = True
        if in_proc and re.match('var',line):
            raise Exception('Define variables inside a procedure.')
        if count < 0: raise Exception('End before beginning a block.')
    if count != 0: raise Exception('Unclosed blocks.')
    #remove procedures from code
    procs,code = get_procs(code)
    #check for loops in procedures
    proc_links = {}
    for name,p in procs.items():
        calls = [re.match(f'call (?P<n>{VAR})',line).group('n') for line in p['body'] if re.match(f'call (?P<n>{VAR})',line)]
        proc_links[name] = calls
    #implement cycle finding code here:
    def isCyclic(graph): 
        def _isCyclic(node, visited, recStack): 
            visited[node] = True
            recStack[node] = True
            for neighbour in graph[node]: 
                if visited[neighbour] == False: 
                    if _isCyclic(neighbour, visited, recStack): 
                        return True
                elif recStack[neighbour]: 
                    return True
            recStack[node] = False
            return False
        vis = {i:False for i in graph}
        rec = {i:False for i in graph}
        for n in graph: 
            if vis[n] == False: 
                if _isCyclic(n,vis,rec): 
                    return True
        return False
    if isCyclic(proc_links): raise Exception("Recursive call.")
    
    code = expand_calls(code, procs)
    code = [line for line in code if line.strip()]
    inst = {"inc|dec|set":['v','vn'],"add|sub|mul|div|mod|cmp":['vn','vn','v'],
            "divmod":['vn','vn','v','v'], "a2b":['vn','vn','vn','v'],"b2a":['vn','v','v','v'],
            "lset":['l','vn','vn'], 'lget':['l','vn','v'],'ifeq|ifneq|wneq':['v','vn'],"end":[],
            "read":['v'],"msg":['vs']}
    vars_defd = []
    lists_defd = []
    for line in code:
        if line[:4] == 'var ':
            vs = [x.group() for x in re.finditer(VAR_SINGLE, line[4:])]
            for v in vs:
                vn = re.split(' |\\[',v.strip())[0]
                if vn in vars_defd or vn in lists_defd:
                    raise Exception('Duplicate var names.')
                if '[' in v: lists_defd.append(vn)
                else: vars_defd.append(vn)
            continue
        for command in inst:
            m = re.match(f'(?:{command})( |$)', line)
            if m:
                params = list(re.finditer(PARAM, line[m.span()[1]:]))
                if m.group() == 'msg ':
                    for p in params:
                        if not (p.group('v') or p.group('s')):
                            raise Exception('Invalid entry in msg cmd')
                        if p.group('v') and p.group('v') in lists_defd:
                            raise Exception('Expect a variable but got a list')
                        if p.group('v') and not p.group('v') in vars_defd:
                            raise Exception('Undefined var names: ' + p.group('v'))
                    break
                if len(params) is not len(inst[command]):
                    raise Exception('Number of arguments for an instruction does not match the expectation.')
                for p,types in zip(params,inst[command]):
                    ptype = ''
                    v = p.group('v')
                    if v: 
                        if v in lists_defd: ptype = 'l'
                        elif v in vars_defd: ptype = 'v'
                        else: raise Exception('Undefined var names: ' + v)
                    else: ptype = 's' if p.group('s') else 'n'
                    if ptype not in types:
                        if ptype == 'l' and 'v' in types:
                            raise Exception('Expected a variable but got a list')
                        if ptype == 'v' and 'l' in types:
                            raise Exception('Expected a list but got a variable.')
                        raise Exception('Expected a variable but got something else')
                break
        else:
            raise Exception('Unknown instructions: ' + line)
    return code
        
def get_procs(code):
    '''Given code, removes the procs and returns them along with the code sans procs'''
    block_titles = ['ifeq ', 'ifneq ', 'wneq ']
    procs, count, edited_code, proc = {}, 0, [], ""
    for line in code:
        if line[:5] == 'proc ':
            if count is not 0: raise Exception('Nested procedures.')
            count = 1
            #proc detected
            name = re.search(f'(?<=proc )({VAR})',line).group(0)
            ds = r"\$"
            params = re.search(f'(?<=proc {re.sub(ds,ds,name)} )(({VAR})( |\n|$))*',line)
            params = params.group(0).split(' ') if params else []
            if len(params) is not len(set(params)): 
                raise Exception('Duplicate parameter names.')
            if name in procs: raise Exception('Duplicate procedure names.')
            procs[name] = {'params':params, 'body':[]}
            proc = name
            continue
        if any([re.match(i,line) for i in block_titles]) and count > 0: 
            count += 1
        if count > 0: procs[proc]['body'].append(line)
        else: edited_code.append(line)
        if line == 'end' and count > 0: count -= 1
    for p,v in procs.items(): v['body'].pop()
    return procs, edited_code

def expand_calls(code, procs):
    '''Replaces all calls with the body of defined procs'''
    '''THIS FUNCTION ASSUMES  RECURSION HAS ALREADY BEEN CHECKED BY VALIDATE'''
    CALL = f'call (?P<fn>{VAR})(?P<params>(?: {VAR})*)'
    def _expanded_calls(c, p):
        expanded = []
        for line in c:
            call = re.match(CALL, line)
            if call:
                proc = p.get(call.group('fn').strip())
                if not proc: raise Exception('Undefined procedure.')
                params = call.group('params').strip().split(' ')
                if len(params) is not len(proc['params']):
                    raise Exception('The length of arguments does not match the length of parameters.')
                for subline in proc['body']:
                    for i,_p in enumerate(params):
                        old = proc["params"][i]
                        ds = r"\$"
                        subline = re.sub(fr'(?<= ){re.sub(ds,ds,old)}(?=(?: |\n|$))',_p + "~",subline)
                    expanded.append(subline)
                expanded = [re.sub('~','',line) for line in expanded]
            else: expanded.append(line)
        return expanded
    while any(re.match(CALL, line) for line in code):
        code = _expanded_calls(code, procs)
    return code

In [8]:
code = """
#Example to show capabilities of language:
var n1, n2, n3,x,y,z

var input
call read_byte input
call print_fib_to input

#very limited by the storage space given
proc print_fib_to n
    set x 0
    set y 1
    wneq n 0
        call print_byte y
        add x y z
        set x y
        set y z
        dec n 1
    end
end

proc print_byte n
    b2a n n1 n2 n3
    msg n1 n2 n3 " "
end

proc read_byte n
    read n1
    read n2
    read n3
    a2b n1 n2 n3 n
end
"""
print(kcuf(code))

>>>>>>>>>>>,>,>,<<<<<<<<<<<[-]<[-]<[-]>>>>>>>>>>[-]<<<<<<<<[-]>>>>>>>>>[<<<<<<<<<+>>>>>>>>+>-]<[>+<-][-]<<<<<<<<<[-]>>>>>>>>>>>[<<<<<<<<<<<+>>>>>>>>>+>>-]<<[>>+<<-][-]<<<<<<<<<<[-]>>>>>>>>>>>>>[<<<<<<<<<<<<<+>>>>>>>>>>+>>>-]<<<[>>>+<<<-]<<<<<<<<------------------------------------------------<------------------------------------------------<------------------------------------------------>>>>>>>[-]+<[-]++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++>>>[-]<[-]<[>+<-]>[<<[>+>>+<<<-]>>>[<<<+>>>-]<-]<<[-]>>>[-]<[-]<[>+<-]>[<<<<<<[>>>>>+>>+<<<<<<<-]>>>>>>>[<<<<<<<+>>>>>>>-]<-]<<<<<<[-]>>>>>>>>[-]<<<<<<<<[-]>>>>>[<<<<<+>>>>>>>>+<<<-]>>>[<<<+>>>-]<<<[-][-]+<[-]++++++++++>>>[-]<[-]<[>+<-]>[<<[>+>>+<<<-]>>>[<<<+>>>-]<-]<<[-]>>>[-]<[-]<[>+<-]>[<<<<<<<[>>>>>>+>>+<<<<<<<<-]>>>>>>>>[<<<<<<<<+>>>>>>>>-]<-]<<<<<<<[-]>>>>>>>>>[-]<<<<<<<<<[-]>>>>>>[<<<<<<+>>>>>>>>>+<<<-]>>>[<<<+>>>-]<<<[-]>>>[-]<<<<<<<<[<<+>>>>>>>>>>+<<<<<<<<-]>>>>>>>>[<<<<<<<<+>>>>>